## Preparation
### Import

In [10]:
import numpy as np
import pandas as pd
# %pip install -q amplpy gspread --upgrade
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin","highs"],  # modules to install
    license_uuid="f86e025b-7515-40af-9caf-d3bdff5ec30d",  # license to use
)  # instantiate AMPL object and register magics


Licensed to 60-day trial license.


In [11]:
%%ampl_eval
option version;

option version 'AMPL Development Version 20240606 (MSVC 19.40.33811.0, 64-bit)\
Licensed to 60-day trial license.\
Temporary license expires 20240803.\
Using license file "c:\Users\ASUS\miniconda3\envs\dunnbebes\lib\site-packages\ampl_module_base\bin\ampl.lic".\
';


### Load Data

In [ ]:
"""Parameter"""
# Demand
demand    = pd.read_excel("small_data.xlsx", sheet_name="Demand")
ProductID = sorted(demand['ProductID'].unique().tolist())
ShipToID  = sorted(demand['ShipToID'] .unique().tolist())

demand_df = (
    demand.groupby(["ShipToID", "ProductID"])["DemandInKG"]
    .sum()
    .reset_index()
)

full_grid   = pd.MultiIndex.from_product([ShipToID, ProductID], names=["ShipToID", "ProductID"]).to_frame(index=False)
demand_df   = full_grid.merge(demand_df, on=["ShipToID", "ProductID"], how="left").fillna(0)
demand_dict = {(row['ShipToID'], row['ProductID']): round(row['DemandInKG'],4) for index, row in demand_df.iterrows()}

# Supply
supply      = pd.read_excel("DATA DNO\small_data.xlsx", sheet_name="Supply")
SupplierID  = sorted(supply['SupplierID'] .unique().tolist())

full_grid   = pd.MultiIndex.from_product([SupplierID, ProductID], names=["SupplierID", "ProductID"]).to_frame(index=False)
supply_df   = full_grid.merge(supply, on=["SupplierID", "ProductID"], how="left").fillna(0)
supplyCap_dict = {(row['SupplierID'], row['ProductID']): round(row['SupplierCapInKG'],4) for index, row in supply_df.iterrows()}


# DC capacity
facility    = pd.read_excel("DATA DNO\small_data.xlsx", sheet_name="Facility")
FacilityID  = sorted(facility['FacilityID'] .unique().tolist())
facilityCapKG_dict  = {(row['FacilityID']): round(row['FacilityCapInKG'],4) for index, row in facility.iterrows()}
# facilityCapCBM_dict = {(row['FacilityID']): round(row['FacilityCapInCBM'],4) for index, row in facility.iterrows()}


"""Coefficients"""
# Transportation Cost
## Purchase 

## Transfer

## To Customer

# Inventory Cost

# Customer Loss




'Coefficients'

## Model
### Mathematical model

In [15]:
%%writefile shipment.mod
reset;

# Sets
set SupplierID;
set FacilityID;
set ShipToID;
set ProductID;

# Parameters
param DemandInKG             {ShipToID,   ProductID}             >= 0 default 0;
## Capacity
param SupplierCapInKG        {SupplierID, ProductID}             >= 0 default 0;
param FacilityCapInKG        {FacilityID}                        >= 0 default 0;
# param FacilityCapInCBM       {FacilityID}                        >= 0 default 0;
# ## Conversion rate
# param CR_kg_to_CBM           {ProductID}                         >= 0 default 0;
# ## Cost
# param OrderingCost           {SupplierID, FacilityID}            >= 0 default 0;
# param TransportCost_transfer {FacilityID, FacilityID}            >= 0 default 0;
# param TransportCost_toShipTo {FacilityID, ShipToID}              >= 0 default 0;
# param RentingCost            {FacilityID}                        >= 0 default 0;
# param HandlingCost           {FacilityID}                        >= 0 default 0;
# param CustomerLoss                                               >= 0 default 0;



# Variables
var MetDemand     {ShipToID, ProductID}                                       >= 0;
var UnmetDemand   {ShipToID, ProductID}                                       >= 0;
var Purchase      {SupplierID, FacilityID, ProductID}                         >= 0;
var InventoryFac1 {FacilityID, ProductID}                                     >= 0;
var InventoryFac2 {FacilityID, ProductID}                                     >= 0;
var TransferIn    {originFac in FacilityID, destFac in FacilityID, ProductID} >= 0;
var TransferOut   {originFac in FacilityID, destFac in FacilityID, ProductID} >= 0;
var ServeCustomer {FacilityID, ShipToID, ProductID}                           >= 0;

# Objective
minimize Cost:
    # sum {sup in SupplierID, fac in FacilityID, product in ProductID}                  Purchase [sup, fac, product] * OrderingCost [sup, fac] +
    # sum {originfac in FacilityID, destinationfac in FacilityID, product in ProductID} TransferOut
    
    sum {shipto in ShipToID, product in ProductID} UnmetDemand[shipto, product];

# Constraints
## Demand
subject to Demand_Balance {shipto in ShipToID, product in ProductID}:
    MetDemand[shipto, product] + UnmetDemand[shipto, product] = DemandInKG[shipto, product];

subject to ServeCustomer_Balance {shipto in ShipToID, product in ProductID}:
    MetDemand[shipto, product] = sum {fac in FacilityID} ServeCustomer[fac, shipto, product];

## Supply Capacity
subject to Supply_Capacity {sup in SupplierID, product in ProductID}:
    sum {fac in FacilityID} Purchase[sup, fac, product] <= SupplierCapInKG[sup, product];

## Flow Balancing
subject to Flow_Balance_Purchase {fac in FacilityID, product in ProductID}:
    InventoryFac1[fac, product] = sum {sup in SupplierID} Purchase[sup, fac, product];

subject to Flow_Balance_Transfer {fac in FacilityID, product in ProductID}:
    InventoryFac2[fac, product] = InventoryFac1[fac, product]
                                   - sum {destinationFac in FacilityID} TransferOut[fac, destinationFac, product]
                                   + sum {originFac in FacilityID} TransferIn[originFac, fac, product];

subject to Flow_Balance_Transfer {fac in FacilityID, product in ProductID}:
    sum {destinationFac in FacilityID} TransferOut[fac, destinationFac, product]
                                   + sum {originFac in FacilityID} TransferIn[originFac, fac, product];

subject to Flow_Balance_Inventory {fac in FacilityID, product in ProductID}:
    sum {shipto in ShipToID} ServeCustomer[fac, shipto, product] <= InventoryFac2[fac, product];

## Facility Capacity
subject to Facility_Capacity_1_inKG {fac in FacilityID}:
    sum {product in ProductID} InventoryFac1[fac, product] <= FacilityCapInKG[fac];

subject to Facility_Capacity_2_inKG {fac in FacilityID}:
    sum {product in ProductID} InventoryFac2[fac, product] <= FacilityCapInKG[fac];

subject to Facility_Capacity_1_inCBM {fac in FacilityID}:
    sum {product in ProductID} InventoryFac1[fac, product]*CR_kg_to_CBM[product] <= FacilityCapInCBM[fac];

subject to Facility_Capacity_2_inCBM {fac in FacilityID}:
    sum {product in ProductID} InventoryFac2[fac, product]*CR_kg_to_CBM[product] <= FacilityCapInCBM[fac];

Overwriting shipment.mod


### Load Set and Parameters

In [16]:
ampl.read('shipment.mod')

# Load data into ampl
ampl.set['ShipToID']                    = ShipToID
ampl.set['ProductID']                   = ProductID
ampl.set['SupplierID']                  = SupplierID
ampl.set['FacilityID']                  = FacilityID


ampl.param['DemandInKG']                = demand_dict
## Capacity
ampl.param['SupplierCapInKG']           = supplyCap_dict
ampl.param['FacilityCapInKG']           = facilityCapKG_dict
# ampl.param['FacilityCapInCBM']          = facilityCapCBM_dict
# ## Conversion
# ampl.param['CR_kg_to_CBM']              = CR_kg_to_CBM_dict
# ## Cost
# ampl.param['PurchasingCost']            = CR_kg_to_CBM_dict
# ampl.param['TransportCost_transfer']    = CR_kg_to_CBM_dict
# ampl.param['TransportCost_toShipTo']    = CR_kg_to_CBM_dict
# ampl.param['RentingCost']               = CR_kg_to_CBM_dict
# ampl.param['HandlingCost']              = CR_kg_to_CBM_dict
# ampl.param['CustomerLoss']              = CR_kg_to_CBM_dict


### Solve

In [17]:
%%ampl_eval
option solver highs;
solve;
display MetDemand, UnmetDemand;

HiGHS 1.7.0:HiGHS 1.7.0: optimal solution; objective 0
6 simplex iterations
0 barrier iterations
:                  MetDemand UnmetDemand    :=
2002 '102371 - AU'      0          0
2002 '102372 - AU'     20          0
2003 '102371 - AU'     10          0
2003 '102372 - AU'     30          0
2004 '102371 - AU'      0          0
2004 '102372 - AU'     14          0
;



# Output

In [18]:
"""Demand satisfaction"""
MetDemand   = ampl.var['MetDemand']  .to_pandas()
UnmetDemand = ampl.var['UnmetDemand'].to_pandas()

result1 = MetDemand.reset_index()
result1.columns = ['ShipToID', 'ProductID','MetDemand']

result2 = UnmetDemand.reset_index()
result2.columns = ['ShipToID', 'ProductID','UnmetDemand']

demand_output = pd.merge(result1, result2, on=['ShipToID', 'ProductID'], how='outer')
demand_output = pd.merge(demand_output, demand_df, on=['ShipToID', 'ProductID'], how='outer')



In [19]:
print(demand_output)

   ShipToID    ProductID  MetDemand  UnmetDemand  DemandInKG
0      2002  102371 - AU          0            0         0.0
1      2002  102372 - AU         20            0        20.0
2      2003  102371 - AU         10            0        10.0
3      2003  102372 - AU         30            0        30.0
4      2004  102371 - AU          0            0         0.0
5      2004  102372 - AU         14            0        14.0


In [ ]:
result            = pd.ExcelWriter('DATA DNO\checking.xlsx', engine='xlsxwriter')
demand_output  .to_excel(result, sheet_name='Demand Balance', index=False)
# supply_output  .to_excel(result, sheet_name='Supply Balance', index=False)
result.close() 
